In [ ]:
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from matplotlib import pyplot as plt

In [ ]:
#TO-DO: rewrite red and green mask into the same method.
def red_mask(img, bandwidth, min_sat, min_val):
    half_band = bandwidth // 2
    hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)
    l = np.array([0, min_sat, min_val])
    u = np.array([0 + half_band, 255, 255])
    m1 = cv.inRange(hsv, l, u)
    l = np.array([180 - half_band, min_sat, min_val])
    u = np.array([180, 255, 255])
    m2 = cv.inRange(hsv, l, u)
    return m1 + m2

def green_mask(img, bandwidth, min_sat, min_val):
    half_band = bandwidth // 2
    hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)
    l = np.array([60, min_sat, min_val])
    u = np.array([60 + half_band, 255, 255])
    m1 = cv.inRange(hsv, l, u)
    l = np.array([60 - half_band, min_sat, min_val])
    u = np.array([60, 255, 255])
    m2 = cv.inRange(hsv, l, u)
    return m1 + m2

def dilate(img, size):
    element = cv.getStructuringElement(
        cv.MORPH_ELLIPSE, 
        (2 * size + 1, 2 * size + 1), 
        (size, size)
    )
    return cv.dilate(gray, element)

In [ ]:
ecg = cv.imread("../ecg/normal_1.png")
ecg = cv.cvtColor(ecg, cv.COLOR_BGR2RGB) 
ecg = cv.resize(ecg, (ecg.shape[1] * 5, ecg.shape[0] * 5), interpolation = cv.INTER_AREA) 

In [ ]:
ecg.shape

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(ecg, aspect='equal')

In [ ]:
mask = red_mask(ecg, 32, 31, 96)
plt.figure(figsize=(12, 12))
plt.imshow(mask, aspect='equal')

In [ ]:
mask2 = green_mask(ecg, 32, 31, 96)
plt.figure(figsize=(12, 12))
plt.imshow(mask2, aspect='equal')

In [ ]:
gray = cv.cvtColor(ecg, cv.COLOR_RGB2GRAY) 
ret, gray = cv.threshold(gray, 127, 255, cv.THRESH_BINARY_INV)
gray[gray > 1] = 255
plt.figure(figsize=(12, 12))
plt.imshow(gray, aspect='equal', cmap='gray')

In [ ]:
gray[np.where(mask > 1)] = 0
gray[np.where(mask2 > 1)] = 0
plt.figure(figsize=(12, 12))
plt.imshow(gray, aspect='equal', cmap='gray')

In [ ]:
dilated = dilate(gray, 30)
plt.figure(figsize=(12, 12))
plt.imshow(dilated, aspect='equal', cmap='gray')

In [ ]:
gray = dilate(gray, 1)
plt.figure(figsize=(12, 12))
plt.imshow(gray, aspect='equal', cmap='gray')

In [ ]:
contours, hierarchy = cv.findContours(dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
contoured = ecg.copy()
min_area = (gray.shape[0] * gray.shape[1]) / 100
rects = []
for i, c in enumerate(contours):
    x, y, w, h = cv.boundingRect(c)
    if w * h > min_area:
        rects.append((x, y, w, h))
        contoured = cv.rectangle(contoured, (x, y), (x + w, y + h), (0, 0, 255), 3)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(contoured, aspect='equal', cmap='gray')

In [ ]:
lines = []
for rect in rects:
    x, y, w, h = rect
    lines.append(gray[y:y + h, x:x + w])

In [ ]:
for line in lines:
    plt.figure(figsize=(16, 16))
    plt.imshow(line, aspect='equal', cmap='gray')
    plt.show()

In [ ]:
def line_to_data(line):
    vert = line.shape[0]
    last = 0.5
    result = []
    for i in range(line.shape[1]):
        points = np.where(line[:,i] > 0)[0]
        if points.size == 0:
            result.append(last)
        elif points.size == 1:
            result.append(1 - points[0] / vert)
        else:
            result.append(1 - (points[0] + points[-1]) / 2 / vert)
        last = result[-1]
    result = np.array(result)
    result = result - result.min()
    result = result * (1 / result.max())
    return result

def fold(signal, period):
    signal = signal - signal.min()
    signal = signal * (1 / signal.max())
    signal = signal - signal.mean()
    min_period, max_period = period
    best_fold_score = 0
    scores = []
    folds = []
    for p in range(min_period, max_period):
        folded = np.zeros(p)
        for i in range(signal.size):
            folded[i % folded.size] = folded[i % folded.size] + signal[i]
        folded = folded / (signal.size / p)
        score = np.abs(folded).sum() / folded.size
        if score > best_fold_score:
            best_fold = folded
            best_fold = best_fold - best_fold.min()
            best_fold = best_fold * (1 / best_fold.max())
            best_fold_score = score
            scores.append([p, score])
            folds.append(best_fold)
    scores = np.array(scores)
    x = scores[:,0]
    y = scores[:,1]
    z = np.polyfit(x, y, 2)
    f = np.poly1d(z)
    positive_slope = np.vectorize(lambda l: f(l + 10e-9) > f(l))
    diff = y.copy()
    diff[positive_slope(x) == False] = -10e9
    best_index = np.where(diff == diff.max())[0][0]
    return folds[best_index], scores, f

In [ ]:
for line in lines:
    data = line_to_data(line)
    non_fourier, scores, polyfit = fold(data, (data.size // 128, data.size // 2))
    denoised_data = fourier_denoise(data, f=0.8)
    folded, scores, polyfit = fold(denoised_data, (data.size // 128, data.size // 2))

    #plt.figure(figsize=(18, 2))
    #plt.imshow(line, aspect='equal', cmap='gray')
    #plt.show()

    plt.figure(figsize=(18, 2))
    plt.plot(np.arange(denoised_data.size), denoised_data, c="black")
    plt.show()

    plt.figure(figsize=(18, 2))

    ax = plt.subplot(1, 4, 1)
    ax.plot(np.arange(non_fourier.size), non_fourier, c="black")

    ax = plt.subplot(1, 4, 2)
    ax.plot(np.arange(folded.size), folded, c="black")

    cs = min(non_fourier.size, folded.size)
    ax = plt.subplot(1, 4, 3)
    ax.set_ylim(-0.5, 0.5)
    ax.plot(np.arange(cs), non_fourier[:cs] - folded[:cs], c="black")

    x = scores[:,0]
    y = scores[:,1]
    x_new = np.linspace(x[0], x[-1], 50)
    y_new = polyfit(x_new)
    positive_slope = np.vectorize(lambda l: polyfit(l + 10e-9) > polyfit(l))
    diff = y.copy()
    diff[positive_slope(x) == False] = -10e9

    ax = plt.subplot(1, 4, 4)
    ax.plot(x[diff != diff.max()], y[diff != diff.max()], 'x', x_new, y_new, color="black")
    ax.plot(x[diff == diff.max()], y[diff == diff.max()], 'o', color="red")
    ax.set_xlim([x[0]-1, x[-1] + 1 ])
    plt.show()




In [ ]:
data = line_to_data(lines[0])    
non_fourier, scores, polyfit = fold(data, (data.size // 128, data.size // 10))

x = scores[:,0]
y = scores[:,1]
f = polyfit
x_new = np.linspace(x[0], x[-1], 50)
y_new = f(x_new)

diff = (y - f(x)) * y
best_index = np.where(diff == diff.max())[0][0]
good = diff == diff.max()
bad = diff != diff.max()
plt.plot(x[good], y[good], 'o', color="black")
plt.plot(x[bad], y[bad], 'x', x_new, y_new, color="black")

plt.xlim([x[0]-1, x[-1] + 1 ])
plt.show()

In [ ]:
diff = y - f(x)
best_index = np.where(diff == diff.max())[0][0]

In [ ]:
def fit_curve(x, y):
    z = np.polyfit(x, y, 2)
    f = np.poly1d(z)
    x_new = np.linspace(x[0], x[-1], 50)
    y_new = f(x_new)
    plt.plot(x, y, 'o', x_new, y_new)
    plt.xlim([x[0]-1, x[-1] + 1 ])
    plt.show()

In [ ]:
def fourier_denoise(data, f=0.4):
    ft = np.fft.fft(data)
    freq = np.fft.fftfreq(data.size)    
    threshold = np.abs(freq).max() * f
    ft[np.abs(freq) > threshold] = 0
    ift = np.fft.ifft(ft)
    return ift.real

In [ ]:
def resampled_fourier2d(data, f=10):
    data = np.interp(np.arange(0, data.size, data.size / f), np.arange(0, data.size), data)
    ft = np.fft.fft(data)
    freq = np.fft.fftfreq(data.size)
    pr, pi = np.meshgrid(ft.real, ft.imag)
    plane = np.sqrt(pr ** 2 + pi ** 2)
    plane = np.log(plane)
    plane = plane - plane.min()
    plane = plane / plane.max()
    return plane, freq

In [ ]:
def plot_fourier3d(data, f=10):
    plane, freq = resampled_fourier2d(data, f)
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.contourf(np.arange(freq.size), np.arange(freq.size), plane, cmap="cool")
    plt.show() 